In [1]:
from sgp4.api import Satrec
from sgp4 import exporter
import spiceypy as pyspice
import pykep as pk
import numpy as np
import time

In [2]:
line1 = '1 25544U 98067A   19343.69339541  .00001764  00000-0  38792-4 0  9991'
line2 = '2 25544  51.6439 211.2001 0007417  17.6667  85.6398 15.50103472202482'
udpla = pk.udpla.tle(line1, line2)
mjd2000s = np.array([when%1000.2 for when in range(20000)])
satellite = Satrec.twoline2rv(line1, line2)
pla = pk.planet(udpla)

In [3]:
t1 = time.perf_counter(), time.process_time()
for mjd2000 in mjd2000s:
    r = udpla.eph(mjd2000)
t2 = time.perf_counter(), time.process_time()
print(f"udpla.eph")
udplaeph = t2[0] - t1[0]
print(f" Real time: {udplaeph:.5f} seconds")


udpla.eph
 Real time: 0.02139 seconds


In [4]:
t1 = time.perf_counter(), time.process_time()
for mjd2000 in mjd2000s:
    r = pla.eph(mjd2000)
t2 = time.perf_counter(), time.process_time()
print(f"pla.eph")
plaeph = t2[0] - t1[0]
print(f" Real time: {plaeph:.5f} seconds")

pla.eph
 Real time: 0.05173 seconds


In [5]:
t1 = time.perf_counter(), time.process_time()
r = pla.eph_v(mjd2000s)
t2 = time.perf_counter(), time.process_time()
plaephv = t2[0] - t1[0]
print(f"pla.eph_v")
print(f" Real time: {plaephv:.5f} seconds")

pla.eph_v
 Real time: 0.01665 seconds


In [6]:
t1 = time.perf_counter(), time.process_time()
for mjd2000 in mjd2000s:
    jd = mjd2000 + 2451544.5
    jd_i = int(jd)
    jd_fr = jd-jd_i
    e, r, v = satellite.sgp4(jd_i, jd_fr)
t2 = time.perf_counter(), time.process_time()
sgp4p = t2[0] - t1[0]
print(f"sgp4")
print(f" Real time: {sgp4p:.5f} seconds")


sgp4
 Real time: 0.01831 seconds


In [7]:
t1 = time.perf_counter(), time.process_time()
jds = [mjd2000 + 2451544.5 for mjd2000 in mjd2000s]
jd_is = [int(item) for item in jds]
jd_frs = [a-b for a,b in zip(jds, jd_is)]
e, r, v = satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))
t2 = time.perf_counter(), time.process_time()
sgp4pv = t2[0] - t1[0]
print(f"sgp4 - vectorized")
print(f" Real time: {sgp4pv:.5f} seconds")

sgp4 - vectorized
 Real time: 0.01040 seconds


In [8]:
# Summary
print(f"udpla.eph -> {udplaeph:.5f}")
print(f"pla.eph -> {plaeph:.5f}")
print(f"eph_v.eph -> {plaephv:.5f}")
print(f"sgp4 -> {sgp4p:.5f}")
print(f"sgp4_v -> {sgp4pv:.5f}")

udpla.eph -> 0.02139
pla.eph -> 0.05173
eph_v.eph -> 0.01665
sgp4 -> 0.01831
sgp4_v -> 0.01040


In [9]:
pk.utils.load_spice_kernels("../../pykep/data/de440s.bsp")
udpla = pk.udpla.spice("JUPITER BARYCENTER", "ECLIPJ2000", "SSB")
pla = pk.planet(udpla)

In [10]:
t1 = time.perf_counter(), time.process_time()
for mjd2000 in mjd2000s:
    r = udpla.eph(mjd2000)
t2 = time.perf_counter(), time.process_time()
print(f"udpla.eph")
udplaeph = t2[0] - t1[0]
print(f" Real time: {udplaeph:.5f} seconds")

udpla.eph
 Real time: 0.11554 seconds


In [11]:
t1 = time.perf_counter(), time.process_time()
for mjd2000 in mjd2000s:
    r = pla.eph(mjd2000)
t2 = time.perf_counter(), time.process_time()
print(f"pla.eph")
plaeph = t2[0] - t1[0]
print(f" Real time: {plaeph:.5f} seconds")

pla.eph
 Real time: 0.16039 seconds


In [12]:
t1 = time.perf_counter(), time.process_time()
r = pla.eph_v(mjd2000s)
t2 = time.perf_counter(), time.process_time()
plaephv = t2[0] - t1[0]
print(f"pla.eph_v")
print(f" Real time: {plaephv:.5f} seconds")

pla.eph_v
 Real time: 0.05030 seconds


In [13]:
t1 = time.perf_counter(), time.process_time()
for mjd2000 in mjd2000s:
    rv, _ = pyspice.spkezr("JUPITER BARYCENTER", (mjd2000-0.5)*pk.DAY2SEC, "ECLIPJ2000", "NONE", "SSB")
t2 = time.perf_counter(), time.process_time()
print(f"spice")
spicep = t2[0] - t1[0]
print(f" Real time: {spicep:.5f} seconds")


spice
 Real time: 0.07272 seconds


In [14]:
t1 = time.perf_counter(), time.process_time()
rv, _ = pyspice.spkezr("JUPITER BARYCENTER", (mjd2000s-0.5)*pk.DAY2SEC, "ECLIPJ2000", "NONE", "SSB")
t2 = time.perf_counter(), time.process_time()
print(f"spicev")
spicepv = t2[0] - t1[0]
print(f" Real time: {spicepv:.5f} seconds")


spicev
 Real time: 0.03875 seconds


In [15]:
# Summary
print(f"udpla.eph -> {udplaeph:.5f}")
print(f"pla.eph -> {plaeph:.5f}")
print(f"eph_v.eph -> {plaephv:.5f}")
print(f"spice -> {spicep:.5f}")
print(f"spice_v -> {spicepv:.5f}")

udpla.eph -> 0.11554
pla.eph -> 0.16039
eph_v.eph -> 0.05030
spice -> 0.07272
spice_v -> 0.03875
